# 4. Managing Extreme Risks

In [5]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import time
import re
pd.set_option('display.max_row', None)

In [6]:
########################################
########### IMPORT DATA ################
########################################

# Import drug data using pandas
data = pd.read_csv('drugs.csv', index_col=0)
projects = data.columns
t_area = data.iloc[0]
ttm = data.iloc[1]
ret = data.iloc[2]
cost = data.iloc[3]

#import covariance matrix
cov=pd.read_csv('drugs_cov.csv', index_col=0)

#therapeutic areas
ther=[
"Oncology", "Cardiovascular", "Respiratory and dermatology", "Transplantation",
"Rheumatology and hormone therapy", "Central nervous system", "Ophtalmics"]

#budget constraints for therapeutic areas
t_bud={"Oncology": 100,
       "Cardiovascular": 200,
       "Respiratory and dermatology": 150,
       "Transplantation": 100,
       "Rheumatology and hormone therapy": 300,
       "Central nervous system": 100,
       "Ophtalmics": 50}

interest_rate=0.03
L = np.linalg.cholesky(cov)

In [24]:
########################################
########### MODEL ######################
########################################

# Create an empty model
m = gp.Model('portfolio')

# ADD DECISION VARIABLES HERE
vars = pd.Series(m.addVars(projects, vtype=GRB.BINARY, name="projects"))
h = m.addVar(name="h")
c = m.addVars(114, name="c")

########################################
#### CONSTRAINTS & OBJ FUNCTIONS #######
########################################

# ADD CONSTRAINTS HERE

# Maximum Cost by Therapeutic Area
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == "Oncology") <= 100)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Cardiovascular') <= 200)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Respiratory and dermatology') <= 150)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Transplantation') <= 100)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Rheumatology and hormone therapy') <= 300)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Central nervous system') <= 100)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Ophtalmics') <= 50)


# Pipeline Constraints
m.addConstr(sum(vars[t] for t in range(len(projects)) if ttm[t] in ["1"]) >= 0.15*sum(vars[t] for t in range(len(projects))))
m.addConstr(sum(vars[t] for t in range(len(projects)) if ttm[t] in ["2", "3"]) >= 0.2*sum(vars[t] for t in range(len(projects))))
m.addConstr(sum(vars[t] for t in range(len(projects)) if ttm[t] in ["4", "5"]) >= 0.25*sum(vars[t] for t in range(len(projects))))

#########################
# 95% VAR


m.addConstr(((ret.T @ vars + (1000-(cost.T @ vars))*(1+interest_rate)) - 1.645*h) >= 15780.6902)
m.addConstr(h**2 >= sum(c[i]**2 for i in range(len(projects))))

for i in range(114):
    val = sum(L.T[i][j] * vars[j] for j in range(114))
    m.addConstr(c[i] == val)
    
m.addConstr(h >= 0)

#########################

# ADD OBJECTIVE FUNCTION HERE
obj = ret.T @ vars + (1000-(cost.T @ vars))*(1+interest_rate)
#obj = sum(ret[i]*vars[i] for i in range(len(projects)))

m.setObjective(obj, GRB.MAXIMIZE)
#m.setParam('OutputFlag', 0)
m.optimize()


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126 rows, 229 columns and 1729 nonzeros
Model fingerprint: 0xd4aa0de9
Model has 1 quadratic constraint
Variable types: 115 continuous, 114 integer (114 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e-01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+04]
Presolve removed 8 rows and 7 columns
Presolve time: 0.01s
Presolved: 119 rows, 336 columns, 1829 nonzeros
Presolved model has 114 quadratic constraint(s)
Variable types: 222 continuous, 114 integer (114 binary)

Root relaxation: objective 2.354989e+04, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 23549.8909    0    7          - 23549.8909    

In [22]:
# ADD PRINTING HERE:
# Print optimal value of the objective function
print('\nValue of objective function \nExpected Return: \n$%g' % m.objVal,"(Million)")

# Print optimal values for the decision variables
print('\nDecision variables \nSelected Projects:\n')
for v in m.getVars():
    if "projects" in v.varName:
        if v.x >= 0.5:
            print('%s = %g' % (v.varName, v.x))
           
        
for v in m.getVars():
    if "h" == v.varName:
        print('\nAuxiliary variable\n %s = %g' % (v.varName, v.x))
            
# Print the number of funded projects
temp=0
for v in m.getVars():
    if "projects" in v.varName:
        if v.x>0.5:
            temp=temp+1
print('\nNumber of selected projects:', temp)

# Print total cost (buget spent on projects)
total_cost = round((vars.T @ cost).getValue(), 4)
print('\nTotal cost: $%g' % total_cost , "(Million)")

# Print the proportion of the budget spent on projects
print('\nPercent of the overall budget used for drug development:\n', round(total_cost/10, 4), "%")

# Print the portfolio variance when maximizing return  
portfolio_risk = (np.dot(np.dot(vars.T, cov), vars)).getValue()
print("\nPortfolio variance(risk) at the risk neutral level:", round(portfolio_risk,4))

print("\nextreme risk at 95% VaR\n", (ret.T @ vars + (1000-(cost.T @ vars))*(1+interest_rate)).getValue() - 1.645*sqrt(portfolio_risk))


Value of objective function 
Expected Return: 
$23123 (Million)

Decision variables 
Selected Projects:

projects[3] = 1
projects[4] = 1
projects[6] = 1
projects[13] = 1
projects[17] = 1
projects[18] = 1
projects[20] = 1
projects[21] = 1
projects[22] = 1
projects[24] = 1
projects[25] = 1
projects[27] = 1
projects[28] = 1
projects[29] = 1
projects[30] = 1
projects[39] = 1
projects[40] = 1
projects[42] = 1
projects[43] = 1
projects[44] = 1
projects[45] = 1
projects[47] = 1
projects[48] = 1
projects[50] = 1
projects[57] = 1
projects[58] = 1
projects[62] = 1
projects[66] = 1
projects[69] = 1
projects[72] = 1
projects[76] = 1
projects[77] = 1
projects[78] = 1
projects[86] = 1
projects[91] = 1
projects[98] = 1
projects[99] = 1
projects[101] = 1
projects[102] = 1
projects[104] = 1
projects[105] = 1
projects[106] = 1
projects[109] = 1
projects[110] = 1
projects[111] = 1
projects[112] = 1

Auxiliary variable
 h = 4463.39

Number of selected projects: 46

Total cost: $873.37 (Million)

Percent 

### Analysis on 95% Value at Risk

* min_r = 15771.43981518629 (as the "h" is the auxiliary, the minimum "r" can range from 0 to 15771.4398 which is the 95% VaR where h = stdDev of the selected portfolio)
* max_risk = 20076423.45
* 15771.43981518629
#### ---------------------------------------------------------------------------------------
* max_r = 15780.690284642398 (due to "h", maximum r can be formed like 15771.4398 < r <= 15780.6902)
* min_risk = 19921861.5
* 95% VaR = 15780.690284642398